In [1]:
# from os.path import dirname, realpath, sep, pardir
# import sys
# sys.path.append(dirname(realpath(__file__)) + sep + pardir)

# load env ------------------------------------------------------------------------
import os
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "false"


# debug ------------------------------------------------------------------
from langchain.globals import set_debug, set_verbose
set_verbose(True)
set_debug(False)

from langchain_core.messages import (
    AIMessage, 
    HumanMessage,
    ToolMessage
)
from langgraph.graph import END, StateGraph, START
from agents import(
    AgentState,
    agent_metadata,
    agent_names
)
from tools import all_tools
from chat_history import load_chat_history
from langgraph.checkpoint.memory import MemorySaver

## Define Tool Node
from langgraph.prebuilt import ToolNode
from typing import Literal

tool_node = ToolNode(all_tools)

def router(state) -> Literal["call_tool", "continue", "__end__"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "FINALANSWER" in last_message.content:
        return "__end__"
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    else:
        return "continue"


## Workflow Graph ------------------------------------------------------------------------
workflow = StateGraph(AgentState)

# add agent nodes
for name, value in agent_metadata.items():
    workflow.add_node(name, value['node'])
    
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "crm_agent",
    router,
    {
        "call_tool": "call_tool",
        "__end__": END,
        "continue": END, 
        }
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {name:name for name in agent_names},
)

workflow.add_edge(START, "crm_agent")


def format_chat_history_to_str(history:list[AIMessage|HumanMessage])->str:
    history_str = ""
    for chat in history:
        if isinstance(chat, HumanMessage):
            history_str += "Human: "+ chat.content.strip() + "\n"
        if isinstance(chat, AIMessage):
            history_str += "AI: "+ chat.content[:50].replace("\n","\t").strip() + "...\n"
    
    if history_str:
        return history_str
    else:
        return "--Empty chat history--"
    

def __submitMessage(
    input:str, 
    user_id:str="test", 
    verbose:bool=False,
    recursion_limit:int=10
    ) -> str:
    
    os.environ['CURRENT_USER_ID'] = user_id
    
    # memory only keep chat history only along agents.
    internal_level_memory = MemorySaver()
    graph = workflow.compile(checkpointer=internal_level_memory)

    events = graph.stream(
        {
            "messages": [input]
            # "chat_history": input
        },
        # Maximum number of steps to take in the graph
        {"recursion_limit": recursion_limit, "thread_id":"a"},
    )
    
    if not verbose:
        events = [e for e in events]
        response = list(events[-1].values())[0]
    else:
        for e in events:
            a = list(e.items())[0]
            a[1]['messages'][0].pretty_print()
        
        response = a[1]
    
    response = response["messages"][0].content
    response = utils.format_bot_response(response, markdown=True)
    
    return response


def listening_chat_history_from_db(user_id:str):
    chat_history = load_chat_history(user_id=user_id)
    chat_history = format_chat_history_to_str(chat_history)
    bot_response = __submitMessage(input=chat_history, user_id=user_id, verbose=True)

    return bot_response


def listening_chat_history(chat_history:list[AIMessage|HumanMessage], user_id:str):
    chat_history = format_chat_history_to_str(chat_history)
    bot_response = __submitMessage(input=chat_history, user_id=user_id, verbose=True)

    return bot_response

In [2]:
chat_history = load_chat_history(user_id="Ud1a3d84c7155aea634e3e607960fb6df")
print(format_chat_history_to_str(chat_history))

Human: มีสินค้าอะไรบ้าง
AI: สินค้าที่มีใน Central Mall สาขา Prasert-Manukitch...
Human: มีทั้งกี่สินค้า
AI: สินค้าที่มีใน Central Mall สาขา Prasert-Manukitch...
Human: อยู่ที่ไหน
AI: Central Mall สาขา Prasert-Manukitch ตั้งอยู่ที่:...
Human: จะซื้อทำไง
AI: ในการซื้อสินค้าที่ Central Mall สาขา Prasert-Manuk...
Human: ส่งมีไหม
AI: Central Mall สาขา Prasert-Manukitch มีบริการจัดส่...



In [3]:
listening_chat_history_from_db("U1fd6424c0fbe1ef59efe9eb49fb2552f")

================================== Ai Message ==================================
Name: crm_agent
Tool Calls:
  get_customer_information (call_Fp8Qi16IKIfIMx2SRjPeBfKZ)
 Call ID: call_Fp8Qi16IKIfIMx2SRjPeBfKZ
  Args:
================================= Tool Message =================================
Name: get_customer_information

{'_id': ObjectId('66e90b7ad82ebda0ef578d8a'), 'user_id': 'U1fd6424c0fbe1ef59efe9eb49fb2552f', 'hobbies_interests': 'Shopping, Food', 'name': 'Unknown', 'preferred_brands': 'Unknown', 'preferred_products_categories': 'Groceries, Fresh Produce', 'price_sensitivity': 'Price-sensitive', 'special_occasions': 'None'}
================================== Ai Message ==================================
Name: crm_agent
Tool Calls:
  save_customer_information (call_1jGJAOSibnI24XY0TIqH5A5B)
 Call ID: call_1jGJAOSibnI24XY0TIqH5A5B
  Args:
    input_dict: {'name': 'Unknown', 'age': None, 'gender': None, 'special_occasions': 'None', 'price_sensitivity': 'Price-sensitive', 'hobbie

KeyboardInterrupt: 

In [7]:
chat_history = load_chat_history(user_id="test")
chat_history

[HumanMessage(content='มีสินค้าอะไรบ้าง'),
 AIMessage(content=' สินค้าที่มีใน Central Mall สาขา Prasert-Manukitch มีดังนี้:\n\n1. **Laptop XYZ**\n   - ราคา: 12,000.99 บาท\n   - สต็อก: 5 ชิ้น\n   - หมวดหมู่: อิเล็กทรอนิกส์\n   - รายละเอียด: โน้ตบุ๊กประสิทธิภาพสูงสำหรับการเล่นเกมและทำงาน (แบรนด์: XYZ, รุ่น: XYZ123, รับประกัน: 1 ปี)\n\n2. **Apple**\n   - ราคา: 56 บาท\n   - สต็อก: 50,000 ชิ้น\n   - หมวดหมู่: อาหาร\n   - รายละเอียด: แอปเปิ้ลสดกรอบ (แบรนด์: FarmFresh, รุ่น: Red Delicious)\n\n3. **Smartphone Pro**\n   - ราคา: 56,000 บาท\n   - สต็อก: 150 ชิ้น\n   - หมวดหมู่: อิเล็กทรอนิกส์\n   - รายละเอียด: สมาร์ทโฟนเรือธงที่มีฟีเจอร์ล้ำสมัย (แบรนด์: TechCorp, รุ่น: ProMax, รับประกัน: 2 ปี)\n\n4. **Running Shoes**\n   - ราคา: 8,000 บาท\n   - สต็อก: 90 ชิ้น\n   - หมวดหมู่: เสื้อผ้ากีฬา\n   - รายละเอียด: รองเท้าวิ่งน้ำหนักเบาออกแบบมาเพื่อความสบายในการวิ่งระยะไกล (แบรนด์: RunFast, รุ่น: Speedster, รับประกัน: 1 ปี)\n\n5. **Desk Chair**\n   - ราคา: 15,000 บาท\n   - สต็อก: 13 ชิ้น\n   - หมวดหมู่: เฟ

In [8]:
def format_chat_history_to_str(history:list[AIMessage|HumanMessage]):
    history_str = ""
    for chat in chat_history:
        if isinstance(chat, HumanMessage):
            history_str += "Human: "+ chat.content.strip() + "\n"
        if isinstance(chat, AIMessage):
            history_str += "AI: "+ chat.content[:50].replace("\n","  ").strip() + "...\n"
    
    if history_str:
        return history_str
    else:
        return "--Empty chat history--"
        
print(format_chat_history_to_str(chat_history))

Human: มีสินค้าอะไรบ้าง
AI: สินค้าที่มีใน Central Mall สาขา Prasert-Manukitch...
Human: มีทั้งกี่สินค้า
AI: สินค้าที่มีใน Central Mall สาขา Prasert-Manukitch...
Human: อยู่ที่ไหน
AI: Central Mall สาขา Prasert-Manukitch ตั้งอยู่ที่:...
Human: จะซื้อทำไง
AI: ในการซื้อสินค้าที่ Central Mall สาขา Prasert-Manuk...
Human: ส่งมีไหม
AI: Central Mall สาขา Prasert-Manukitch มีบริการจัดส่...
Human: สวัสดี
AI: สวัสดีครับ/ค่ะ! มีอะไรให้ช่วยเหลือเกี่ยวกับข้อมูล...
Human: ถ้าจะซื้อผักราคาถูกตอนนี้ต้องไปที่ไหนครับ
AI: ขอโทษครับ/ค่ะ แต่ไม่พบข้อมูลเกี่ยวกับร้านค้าที่ขา...
Human: อากาศเป็นยังไงบ้างวันนี้
AI: ขอโทษครับ/ค่ะ ฉันไม่สามารถให้ข้อมูลเกี่ยวกับสภาพอา...
Human: ราคาแอปเปิล
AI: ขอโทษครับ/ค่ะ แต่ไม่พบข้อมูลเกี่ยวกับราคาแอปเปิลใ...
Human: Apple price
AI: I couldn't find any specific information about th...
Human: The best food in thailand
AI: Thailand is renowned for its diverse and flavorfu...
Human: อาหารที่ดีที่สุดในประเทศคืออะไร
AI: ประเทศไทยมีอาหารที่หลากหลายและอร่อยมากมาย ซึ่งบาง...
Human: มีสิ